<a href="https://colab.research.google.com/github/Heulwen31/Artificial-Intelligence-Pac-Man/blob/master/Khoa_luan_tot_nghiep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import open3d as o3d
import numpy as np
import pandas as pd 
import cv2 as cv
import copy
import json
from skimage import transform
import math
import pandas as pd

# CONSTANT

In [ ]:
FILE_IMAGE_2D = "/content/drive/MyDrive/image_2d/anh_chup_q68_mk29.jpg"
FILE_POINT_2D_MAPPING = "/content/drive/MyDrive/point mapping/point_2d.csv"
FILE_POINT_3D_MAPPING = "/content/drive/MyDrive/point mapping/point_3d.csv"
FILE_OBJ = "/content/drive/MyDrive/mesh obj/untitled.obj"

# 2D image processing 

In [ ]:
# read image 2d 
# input path file of 2d image 
# output is value rgb of pixel 
def read_image_2d(file_name):
    return cv.cvtColor(cv.imread(file_name), cv.COLOR_BGR2RGB)/255


# get color from cordinate 2d image 
# input is (x, y) and image matrix of color pixel
# output is rgb value 
# return list [a, b, c]
def get_color_2d_image_pixel(x, y, image, height, width):
    if x < 0 or x > height  or y < 0  or y > width:
        return [1.0, 0.0, 0.0]
    return image[round(y)][round(x)].tolist()


# mapping color to set points mapping 3d to 2d with color from 2d image 
# input :: ls: list point mapping 3d to 2d, image: list color, height: height of image, width: width of image 
# output: list color mapping to 3d
def get_list_color_mapping(ls, image,height, width):
    rs = []
    for point in ls:
        rs.append(get_color_2d_image_pixel(point[0][0], point[0][1], image, height, width))
    return rs

# 3D OBJ processing

In [ ]:
# get list cordinate 3d vertices of OBJ
# input: file path of obj
# output: list vertices mesh
def ls_cordinate_3d_point_vertices(lines):
    result = []
    for line in lines:
        element = line.split()
        if len(element) > 0 and element[0] == 'v':
            result.append([float(element[1]), float(element[2])])
    return result


# add color to verties 
# input: lines is string lines in OBJ file 
# input: ls_vertices_value is list vertices cordinate of OBJ
# output: line with rgb value color.
def add_color_vertices(lines, ls_vertices_value):
    index = 0
    for i in range(len(lines)):
        element = lines[i].split()
        if len(element) > 0 and element[0] == 'v':
            lines[i] = "{} {} {} {}\n".format(lines[i].replace("\n", ""),  ls_vertices_value[index][0], ls_vertices_value[index][1], ls_vertices_value[index][2])
            index += 1
    return lines


# get set point 3d mapping
# input: file path 
# output: list cordinate 3d 
def get_list_3d_point_mapping(lines):
    result = []
    for line in lines:
        temp = line.split(";")
        # result.append([float(temp[2]), float(temp[3]), float(temp[4])])
        result.append([float(temp[2]), float(temp[3])])
    return result


# Transformation matrix 

In [ ]:
def mapping(ls_3d_cordinate, np_src_point_list, np_dest_point_list):
    tform = transform.estimate_transform('projective', np_src_point_list, np_dest_point_list)
    np_bboxes = np.array(ls_3d_cordinate)
    np_dest_bboxes = []
    for poly_points in np_bboxes:
        np_dest_bboxes.append(tform(poly_points))
    return np_dest_bboxes

# Working with file

In [ ]:
# get all lines in file obj have length > 0
def get_lines_file(file_path):
    with open(file_path) as f:
        lines = f.readlines()
        lines = [x for x in lines if len(x) > 0]
        return lines


# write file 
# input: lines with color value in vertices 
# output: result.obj file
def write_add_color_obj(lines) -> None:
    text_file = open("/content/drive/MyDrive/result obj/result.obj", "w")
    for line in lines:
        text_file.write(line)
    text_file.close()

# Mapping Color

# Perpare input

In [ ]:
def get_list_cordinate_2d_point(point_2d):
    result = []
    for i in point_2d["region_shape_attributes"]:
        result.append(list(map(int, i.replace('{"name":"point","cx":', "").replace('"cy":', '').replace("}", "").split(','))))
    return result


def get_list_cordinate_3d_point(point_3d):
    result = []
    for i in point_3d['point']:
        result.append(list(map(float, i.split(';')[2:4])))
    return result


point_2d = pd.read_csv("/content/drive/MyDrive/point mapping/point_2d.csv")
point_3d = pd.read_csv("/content/drive/MyDrive/point mapping/point_3d.csv")

cordinate_2d_point = get_list_cordinate_2d_point(point_2d)
cordinate_3d_point = get_list_cordinate_3d_point(point_3d)


In [ ]:
lines = get_lines_file(FILE_OBJ)

In [ ]:
list_cordinate_3d = ls_cordinate_3d_point_vertices(lines)
print(len(list_cordinate_3d))

3318218


In [ ]:
image = read_image_2d(FILE_IMAGE_2D)

height, width, _ = image.shape

In [ ]:
print(type(lines))

<class 'list'>


# Output

In [ ]:
# mapping_point = mapping(list_cordinate_3d, np.array(cordinate_3d_point),np.array(cordinate_2d_point))
# get_list_color = get_list_color_mapping(mapping_point, image, height, width)
# lines_color = add_color_vertices(lines, get_list_color)
# write_add_color_obj(lines_color)

# Cải thiện thuật toán mapping 


*   Chia nhỏ vùng mapping với việc tính toán ma trận mapping trên một diện tích nhỏ hơn 
*   Điểm tương ứng với vùng nào được ứng với ma trận mapping vùng đó 



In [ ]:
# get cordinate from 3 points (ACB)
def caculate_coner(a, b, c):
    vector_one = [a[0]-c[0], a[1]-c[1]]
    vector_two = [b[0]-c[0], b[1]-c[1]]
    a = 1.0*(vector_one[0]*vector_two[0]+vector_one[1]*vector_two[1])
    b = 1.0*(math.sqrt(vector_one[0]*vector_one[0] + vector_one[1]*vector_one[1])*math.sqrt(vector_two[0]*vector_two[0] + vector_two[1]*vector_two[1]))
    if (a/b) > 1 or (a/b) < -1:
        return math.acos(1)/(2*math.pi)*360
    if b == 0:
        return 0
    return math.acos(a/b)/(2*math.pi)*360

# check point in area 
def check_point_in_area(area, point):
    temp = area.append(area[0])
    total_corner = 0
    for i in range(len(area)-1):
        total_corner += caculate_coner(area[i], area[i+1], point)
    if  round(total_corner) == 360 or round(total_corner) == 180 or round(total_corner) == 90:
        return True
    return False
    
# get mapping matrix 
def mapping_matrix(np_src_point_list, np_dest_point_list):
    tform = transform.estimate_transform('projective', np_src_point_list, np_dest_point_list)
    return tform

# list transform matrix 
def list_mapping_matrix(list_src_point_list, list_dest_point_list):
    result = []
    for i in range(len(list_src_point_list)):
        form = transform.estimate_transform('projective', np.array(list_src_point_list[i]), np.array(list_dest_point_list[i])) 
        result.append(form)
    return result

# mapping point function
def mapping_2(ls_3d_cordinate, list_src_point_list, list_dest_point_list):
    l = len(list_src_point_list)
    np_dest_bboxes = []
    np_bboxes = np.array(ls_3d_cordinate)
    list_mapping_matrix_result = list_mapping_matrix(list_src_point_list, list_dest_point_list)
    for point in np_bboxes:
        check = False
        for i in range(len(list_src_point_list)-1):
            if check_point_in_area(list_src_point_list[i][0:4], point):
                np_dest_bboxes.append(list_mapping_matrix_result[i](point))
                check = True
                break
        if check == False:
                np_dest_bboxes.append(list_mapping_matrix_result[l-1](point))
    return np_dest_bboxes

# cordinate polygon
def set_cordinate_polygon(cordinate_point, set_polygon):
    result = []
    for polygon in set_polygon:
        rs = []
        for i in polygon:
            rs.append(cordinate_point[i])
        result.append(rs)
    result.append(cordinate_point)
    return result

In [ ]:
polygon_1 = [0, 1, 3, 2, 4]
polygon_2 = [2, 3, 6, 5, 7]
polygon_3 = [5, 6, 9, 8, 10]
polygon_4 = [8, 9, 12, 11, 13]
polygon_5 = [11, 12, 15, 14, 16]
polygon_6 = [14, 15, 18, 17, 19]

set_polygon = [polygon_1, polygon_2,polygon_3, polygon_4, polygon_5, polygon_6]
set_cordinate_polygon_2d = set_cordinate_polygon(cordinate_2d_point, set_polygon)
set_cordinate_polygon_3d = set_cordinate_polygon(cordinate_3d_point, set_polygon)

In [ ]:
mapping_point_2 = mapping_2(list_cordinate_3d, set_cordinate_polygon_3d, set_cordinate_polygon_2d)

In [ ]:
color_mapping = get_list_color_mapping(mapping_point_2, image, height, width)

In [ ]:
lines_color = add_color_vertices(lines, color_mapping)
write_add_color_obj(lines_color)